# Azure Quantum Workspace

In [9]:
import qsharp
import qsharp.azure
from azure.quantum import Workspace

workspace = Workspace (
    subscription_id = "83e0e071-c631-4700-bd13-7c6a2545ab1c",
    resource_group = "azurequantum",
    name = "jcrandall-q",
    location = "eastus"
)

targets = qsharp.azure.connect(
   resourceId="/subscriptions/83e0e071-c631-4700-bd13-7c6a2545ab1c/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/jcrandall-q",
   location="eastus")


print("This workspace's targets:")
for target in targets:
    print("-", target.id)


Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace jcrandall-q in location eastus.
This workspace's targets:
- ionq.qpu
- ionq.simulator
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s2-sim
- quantinuum.hqs-lt


# Shor's Algorithm
Given: $M$, the number to be factored:
* Step 1: Pick $n$ qubits so that $N = 2^n > M$.
* Step 2: Pick a random $ a < M $.
* Step 3: If $\text{gcd}(a,M) \neq 1$, we've factored $M$, so stop.
* Step 4: If not, use $a$ and $M$ in shor's circuit and obtain and outcome $|y\rangle$ 
* Step 5: Find the close convergents of $\frac{y}{N}$, and thus a guess for $r$.
* Step 6: See if these two conditions are satisfied:
    * $r$ is even.
    * $a^{\frac{r}{2}} + 1 \equiv_M 0 $ is false
    * If not, try a different Step 2 and repeat steps 3-6.
* Step 7: If the conditions are satisfed, attempt to factor $M$ as shown via the numbers $a^{\frac{r}{2}} + 1$ and $a^{\frac{r}{2}} - 1$.
* Repeat as needed to try different $r$ so that at least one provides a $y$ near $\frac{N}{r}$ 


In [25]:
%%qsharp

open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

// 3 qubit Quantum Fourier Transform
operation Perform3qubitQFT() : Result[] {
        mutable resultArray = [Zero, size = 3];
        use qs = Qubit[3];
        //QFT:
        //first qubit:
        H(qs[0]);
        Controlled R1([qs[1]], (PI()/2.0, qs[0]));
        Controlled R1([qs[2]], (PI()/4.0, qs[0]));
        //second qubit:
        H(qs[1]);
        Controlled R1([qs[2]], (PI()/2.0, qs[1]));
        //third qubit:
        H(qs[2]);

        SWAP(qs[2], qs[0]);

        //Message("Before measurement: ");
        //DumpMachine();
        
        for i in IndexRange(qs) {
            set resultArray w/= i <- M(qs[i]);
        }

        //Message("After measurement: ");
        //DumpMachine();
        
        ResetAll(qs);
        
        return resultArray;
}


In [30]:
Perform3qubitQFT()

[1, 0, 0]